In [ ]:
import re
import csv
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import end_patch as ep #Ram's post patch script
from collections import OrderedDict
sns.set(context = "notebook", style = "darkgrid", font="verdana")

In [ ]:
class Dataframe():
    """Dataframe with initializer and filtering dataframe methods."""
    
    json_col_list = ["date", "name", "container", "patch_duration", "end_patch", "recording.pipetteR", "extraction.endPipetteR", 'extraction.postPatch']
    shiny_col_list = ["sample_id", "cell_name_label", "cluster_label"]
    morph_col_list = ["patched_cell_container", "ims63x_go_no_go", "drivers"]
    
    def __init__(self, file_name):
        """Initializer"""
        self.df = pd.read_csv(file_name)
        
    def filter_df(self, col_list):
        """Filters dataframe based on which columns users wants to display as a table."""
        self.df = self.df[col_list]
        return self.df
    
    def sort_date(df):
        start_date = "2018-01-01"
        end_date = "2018-12-31"

        df = df.set_index("date") #Set date column as index column 
        df = df[start_date:end_date] #Choosing start & end date

        df.sort_index(inplace=True) #Date column as index to sort index column
        df.reset_index(inplace=True) #Date column is not index
        return df
    
    def merge_dfs(df1, df2, df3):
        """Merge three dataframes.

        Args:
            df1: left dataframe
            df2: right dataframe
            df3: right dataframe for merged_df
        Returns:
            merged_df: merged dataframe of df1, df2 and df3

        """

        merged_df = pd.merge(left = df1, right = df2, left_on = "container", right_on = "sample_id", how = "left")
        merged_df = pd.merge(left = merged_df, right = df3, left_on = "container", right_on = "patched_cell_container", how = 'inner')

        merged_df.drop(["sample_id", "patched_cell_container"], axis = 1, inplace = True)
        #merged_df.dropna(subset=["res_index_label", "ims63x_go_no_go"], inplace=True)
        merged_df.dropna(subset=["res_index", "ims63x_go_no_go"], inplace=True)
        merged_df.sort_values("date", inplace = True) #Sorts by date column ascending
        return merged_df
    

In [ ]:
class Json(Dataframe):
    
    def json_pre_filter(self):
        """Filter specifically json dataframe"""
        
        users = "P1|P2|P8|P9|PA|PB|PE|PF"
        
        j.df = j.df[j.df["status"] == "SUCCESS"]
        j.df.dropna(subset = ["container"], inplace = True)
        j.df = j.df[j.df["container"].str.match(users)]

        #Creates patch_duration column and outputs in minutes
        exp_start_time = j.df["recording.timeWholeCellStart"]
        exp_end_time = j.df["extraction.timeRetractionEnd"]
        j.df["patch_duration"] = pd.to_datetime(exp_end_time) - pd.to_datetime(exp_start_time)
        j.df["patch_duration"] = (j.df["patch_duration"].astype("timedelta64[s]"))/60

        j.df = ep.create_end_patch(j.df) #Makes post patch column with Ram's edited version

        j.df["date"] = pd.to_datetime(j.df["date"]) #Converting to YYYY-MM-DD HH:MM:SS
        j.df.sort_values("date", inplace = True) #Sorts date column
        return j.df

In [ ]:
j = Json("jem_metadata.csv")
s = Dataframe("shiny_human.csv")
m = Dataframe("63x.csv")

In [ ]:
json_fill = j.json_pre_filter()
json_df = j.filter_df(Json.json_col_list)

shiny_df = s.filter_df(Dataframe.shiny_col_list)
morph_df = m.filter_df(Dataframe.morph_col_list)

In [ ]:
class Dataframe():
    """Dataframe with initializer and filtering dataframe methods."""
    
    users = "P1|P2|P8|P9|PA|PB|PE|PF"
    acute_col_list = ["container_date", "patched_cell_container", "name", "ri", "upstroke_downstroke_ratio_long_square", "sag"]
    shiny_col_list = ["sample_id", "cell_name_label", "cluster_label"]
    
    def __init__(self, file_name):
        """Initializer"""
        self.df = pd.read_csv(file_name)
        
    def filter_df(self, col_list):
        """Filters dataframe based on which columns users wants to display as a table."""
        self.df = self.df[col_list]
        return self.df
    
    def merge_dfs(self, df1, df2):
        """Merge two dataframes.
        Args:
            df1: left dataframe
            df2: right dataframe
        Returns:
            merged_df: merged dataframe of df1 and df1
        """
        
        merged_df = pd.merge(left = df1, right = df2, left_on = "patched_cell_container", right_on = "sample_id", how = "inner")

        merged_df.drop(columns = ["sample_id", "cell_name_label"], inplace=True)
        merged_df = merged_df[merged_df["patched_cell_container"].str.match(Dataframe.users)]

        #Converting to YYYY-MM-DD
        merged_df["container_date"] = pd.to_datetime(merged_df["container_date"], format = "%y%m%d", errors = "coerce")
        merged_df.sort_values("container_date", inplace = True) #Sorts by date column ascending
        return merged_df

In [ ]:
#Alias for Subclasses
a = Dataframe("human_acute_ephys_features.csv")
s = Dataframe("shiny_human.csv")

#Dataframe based on user choice for columns
acute_df = a.filter_df(Dataframe.acute_col_list)
shiny_df = s.filter_df(Dataframe.shiny_col_list)

#Merged acute and shiny dataframes
merged_df = a.merge_dfs(acute_df, shiny_df)